In [1]:
import re
import pickle
import pandas as pd
import numpy as np
from numpy import nan

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,average_precision_score,recall_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
df= pd.read_csv('Dataset_2738_after_webscraping.csv')  

In [5]:


C = (df['Content']).tolist()

L = (df['Label']).tolist()

In [6]:
#PREPROCESSING
def preprocess(col):
    col_l = []
    for r in col:
        #print(r)
        try:
            a = r.lower() 
            a = a.replace("'s", "")
            #a = a.replace(["\n"], "hjfgjh")
            a = re.sub('[^a-zA-Z0-9 \n\.]', '',a)
            a = re.sub('[:.\n\t]', '',a)
            a = re.sub('[?:!.+,]]][]]//;]', '',a) 
            
            col_l.append(a)
        except:
            #print("No header found")
            col_l.append("No Header")
            pass

    # Downloading punkt and wordnet from NLTK
    nltk.download('punkt')
    print("------------------------------------------------------------")
    nltk.download('wordnet')
    
    wordnet_lemmatizer = WordNetLemmatizer()
    
    nltk.download('stopwords')
    
    lemmatized_text_list = []

    for s in col_l:
        #print(s)
        # Create an empty list containing lemmatized words
        lemmatized_list = []

        # Save the text and its words into an object
        #text = df.loc[row]['Content_Parsed_4']
        text_words = s.split(" ")

        # Iterate through every word to lemmatize
        for word in text_words:
            lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

        # Join the list
        lemmatized_text = " ".join(lemmatized_list)

        # Append to the list containing the texts
        lemmatized_text_list.append(lemmatized_text)
    
    new_lemmatized_text_list = []
    for x in lemmatized_text_list:
        new_lemmatized_text_list.append(" ".join(x.split())) 
    
    pl = pd.Series(new_lemmatized_text_list)
   # pl = pd.Series(col_l)
    stop_words = list(stopwords.words('english'))
    
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        #pl = pd.Series(col_l)
   
        pl = pl.str.replace(regex_stopword, '')
        
    
    return pl.tolist()

In [7]:
con = preprocess(C)
#print(con)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jojy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


------------------------------------------------------------


In [8]:
len(con)

2734

In [9]:
L

['iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-project',
 'iot-pr

In [10]:
label = []

In [11]:
for i in L:
    if i == 'iot-project' or i == 'Iot-Project':
        label.append(1)
        
    elif i == 'non-iot-project':
        label.append(0)

In [12]:
len(label)

2734

In [13]:
category_codes = {'non-iot-project': 0, 'iot-project': 1,}

In [14]:
category_codes

{'non-iot-project': 0, 'iot-project': 1}

In [15]:
la2 = pd.Series(label)

In [16]:
la_2 = la2.replace({'Category_Code':category_codes})

In [17]:
la_2

0       1
1       1
2       1
3       1
4       1
       ..
2729    0
2730    0
2731    0
2732    0
2733    0
Length: 2734, dtype: int64

In [18]:
len(la_2)

2734

In [19]:
 df = pd.DataFrame({"Content":con})

In [20]:
df

,Content
0,connect modbus energy meter arduino monito...
1,remote control laser use multiple argons r...
2,know go check trek get call without stall ...
3,simple tutorial connect microbit azure les...
4,tire joystick control phone use l298n ardui...
...,...
2729,need plant companion dont green thumb tr...
2730,make tiny folcloric breakfast set daughter...
2731,tiny speed challenge inspire make something...
2732,instructable ill show make tiny photo al...


In [21]:
#####################################  Training Module Starts  #########################################

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df, la_2, test_size=0.15, random_state=8)

In [23]:

X_test

,Content
1214,cool iot project beginners use nodemcu bl...
271,weve see float wine bottle holders holder s...
1798,youve ever one iconic rainbow bagels bag...
663,connect particle photon directly azure iot h...
1926,lot part instructable could improve pa...
...,...
636,use two 915mhz radio sendreceive current envi...
1331,fairly straightforward project get introduce ...
392,convert rc car iot car particle photon c...
995,esp32 feather display real time clock read ...


In [24]:
y_train

2489    0
184     1
1360    1
212     0
186     1
       ..
2181    0
2409    0
2033    0
1364    1
451     1
Length: 2323, dtype: int64

In [25]:
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features =300# not clear how to choose this value

In [26]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

In [27]:
features_train = tfidf.fit_transform(X_train['Content']).toarray()
print(features_train)




[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.08142891 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.11856539 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.19864862 ... 0.12465331 0.         0.        ]]


In [28]:
features_test = tfidf.fit_transform(X_test['Content']).toarray()
print(features_test)


[[0.09693965 0.         0.         ... 0.         0.         0.        ]
 [0.         0.08766861 0.09405929 ... 0.         0.09071051 0.09667642]
 [0.10613959 0.06474572 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.06570012 ... 0.         0.         0.        ]
 [0.         0.15130654 0.         ... 0.         0.         0.        ]
 [0.06666594 0.0688545  0.07387371 ... 0.         0.10040958 0.04484501]]


In [29]:


#####################################  Training Module Ends  #########################################

In [30]:
################################## DECISION TREE ###############################################

In [31]:
#DTC =DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,max_features=None, max_leaf_nodes=10, min_samples_leaf=5,min_samples_split=2, min_weight_fraction_leaf=0.0,random_state=None, splitter='random')
DTC = DecisionTreeClassifier(random_state=20)
DTC.fit(features_train, y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=20, splitter='best')

In [32]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(DTC, open(filename, 'wb'))

In [33]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(features_test, y_test)
print("Testing Accuacy",result)

Testing Accuacy 0.9124087591240876


In [34]:

DTC_pred = loaded_model.predict(features_test)
DTC_pred

array([1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,

In [35]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, DTC_pred)
print(matrix)


[[168  36]
 [  0 207]]


In [36]:
report = classification_report(y_test, DTC_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       204
           1       0.85      1.00      0.92       207

    accuracy                           0.91       411
   macro avg       0.93      0.91      0.91       411
weighted avg       0.93      0.91      0.91       411



In [37]:
##############NAIVE BAYES######################

In [38]:
Naive = naive_bayes.MultinomialNB(alpha=100)
Naive.fit(features_train, y_train)



MultinomialNB(alpha=100, class_prior=None, fit_prior=True)

In [39]:
# save the model to disk
filename_naive = 'finalized_naive.sav'
pickle.dump(Naive, open(filename_naive, 'wb'))

In [40]:
# load the model from disk
loaded_naive = pickle.load(open(filename_naive, 'rb'))
result = loaded_naive.score(features_test, y_test)
print("Testing Accuacy",result)

Testing Accuacy 0.9343065693430657


In [41]:

Naive_pred = loaded_naive.predict(features_test)
Naive_pred

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,

In [42]:
matrix = confusion_matrix(y_test, Naive_pred)
print(matrix)
report = classification_report(y_test, Naive_pred)
print(report)


[[199   5]
 [ 22 185]]
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       204
           1       0.97      0.89      0.93       207

    accuracy                           0.93       411
   macro avg       0.94      0.93      0.93       411
weighted avg       0.94      0.93      0.93       411



In [43]:
SVM = svm.LinearSVC(C=0.1)
SVM.fit(features_train, y_train)



LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [44]:
# save the model to disk
filename_svm = 'finalized_svm.sav'
pickle.dump(SVM, open(filename_svm, 'wb'))

In [45]:
# load the model from disk
loaded_svm = pickle.load(open(filename_svm, 'rb'))
result = loaded_svm.score(features_test, y_test)
print("Testing Accuacy",result)

Testing Accuacy 0.9464720194647201


In [46]:
svm_pred = loaded_svm.predict(features_test)
svm_pred

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,

In [47]:
matrix = confusion_matrix(y_test, svm_pred)
print(matrix)
report = classification_report(y_test, svm_pred)
print(report)


[[198   6]
 [ 16 191]]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       204
           1       0.97      0.92      0.95       207

    accuracy                           0.95       411
   macro avg       0.95      0.95      0.95       411
weighted avg       0.95      0.95      0.95       411



In [48]:
knn = KNeighborsClassifier(n_neighbors=9,weights='distance')
knn.fit(features_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='distance')

In [49]:
# save the model to disk
filename_knn = 'finalized_knn.sav'
pickle.dump(knn, open(filename_knn, 'wb'))

In [50]:
# load the model from disk
loaded_knn = pickle.load(open(filename_knn, 'rb'))
result = loaded_knn.score(features_test, y_test)
print("Testing Accuacy",result)

Testing Accuacy 0.902676399026764


In [51]:
knn_pred = loaded_knn.predict(features_test)
knn_pred

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1,

In [52]:
matrix = confusion_matrix(y_test, knn_pred)
print(matrix)
report = classification_report(y_test, knn_pred)
print(report)

[[194  10]
 [ 30 177]]
              precision    recall  f1-score   support

           0       0.87      0.95      0.91       204
           1       0.95      0.86      0.90       207

    accuracy                           0.90       411
   macro avg       0.91      0.90      0.90       411
weighted avg       0.91      0.90      0.90       411



In [53]:
#clf = SGDClassifier(loss="log", penalty="l1", max_iter=100)
SGD = SGDClassifier(random_state=8)
SGD.fit(features_train, y_train)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=8, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [54]:
# save the model to disk
filename_sgd = 'finalized_sgd.sav'
pickle.dump(SGD, open(filename_sgd, 'wb'))

In [55]:
# load the model from disk
loaded_sgd = pickle.load(open(filename_sgd, 'rb'))
result = loaded_sgd.score(features_test, y_test)
print("Testing Accuacy",result)

Testing Accuacy 0.8102189781021898


In [56]:
sgd_pred = loaded_sgd.predict(features_test)
sgd_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,

In [57]:
matrix = confusion_matrix(y_test, sgd_pred)
print(matrix)
report = classification_report(y_test, sgd_pred)
print(report)

[[202   2]
 [ 76 131]]
              precision    recall  f1-score   support

           0       0.73      0.99      0.84       204
           1       0.98      0.63      0.77       207

    accuracy                           0.81       411
   macro avg       0.86      0.81      0.80       411
weighted avg       0.86      0.81      0.80       411



In [58]:
 
random = RandomForestClassifier(n_estimators=8,max_features='log2',random_state=8)
#regressor = RandomForestRegressor(n_estimators=2, max_depth=None,min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None,bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)

random.fit(features_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=8,
                       n_jobs=None, oob_score=False, random_state=8, verbose=0,
                       warm_start=False)

In [59]:
# save the model to disk
filename_random = 'finalized_random.sav'
pickle.dump(random, open(filename_random, 'wb'))

In [60]:
# load the model from disk
loaded_random = pickle.load(open(filename_random, 'rb'))
result = loaded_random.score(features_test, y_test)
print("Testing Accuacy",result)

Testing Accuacy 0.9197080291970803


In [61]:
random_pred = loaded_random.predict(features_test)
random_pred

array([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,

In [62]:
matrix = confusion_matrix(y_test, sgd_pred)
print(matrix)
report = classification_report(y_test, sgd_pred)
print(report)

[[202   2]
 [ 76 131]]
              precision    recall  f1-score   support

           0       0.73      0.99      0.84       204
           1       0.98      0.63      0.77       207

    accuracy                           0.81       411
   macro avg       0.86      0.81      0.80       411
weighted avg       0.86      0.81      0.80       411

